## Boundary matrix, reduction and barcode output (dense matrix)

Creating simplex class and reading function

In [10]:
import numpy as np

class simplex:
    def __init__(self, line):
        self.dim, self.val, self.vert = self.parse_line(line)
    
    def parse_line(self, line):
        splited_line = line.split()
        val = float(splited_line[0])
        dim = int(splited_line[1])
        vert = [int(x) for x in splited_line[2:]]
        assert(len(vert)==dim+1)
        return dim, val, vert

def read_filtration(name):
    simplices = []
    with open(name, "r") as ins:
        for line in ins:
            simplices.append(simplex(line))
    return simplices

Reading the example filtration

In [11]:
simplices = read_filtration('filtration.txt')

Grouping by time

In [12]:
simplices.sort(key=lambda x: x.val)

Sorting the vertices

In [13]:
for s in simplices:
    s.vert.sort()

Creating dictionary for matrix creation

In [14]:
simplices_str =  [str(s.vert).strip('[]') for s in simplices]

d = {}
for i, s in enumerate(simplices_str):
    d[s] = i

Creating dictionary for final output

In [15]:
sDict = {}
for i, s in enumerate(simplices):
    sDict[i] = s

Creating dense matrix

In [17]:
s_matrix = np.zeros((len(simplices),len(simplices)))

for i, s in enumerate(simplices):
    if s.dim>0:
        vs = list(s.vert)
        for v in vs:
            temp = list(vs)
            temp.remove(v)
            #print temp
            s_matrix[d[str(temp).strip('[]')]][i] = 1

Printing matrix

In [18]:
print s_matrix

[[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Reduction

In [19]:
M = len(s_matrix)
low = [float('inf')]*M

for j in range(M):
    for i in range(M):
        auxI = M-i-1
        if s_matrix[auxI][j]==1:
            low[j] = auxI
            if auxI in low[:j]:
                ind = low.index(auxI)
                s_matrix[:,j] = np.logical_xor(s_matrix[:,j],s_matrix[:,ind]).astype(int)
                low[j] = float('inf')
            else:
                break

Printing reduced matrix

In [20]:
print s_matrix

[[ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


Finding barcode

In [21]:
answer = []
for j in range(M):
    if low[j] == float('inf'):
        if j not in low:
            aux = sDict[j]
            answer.append(str(aux.dim)+' '+str(aux.val)+' '+'inf')
    else:
        aux1 = sDict[low[j]]
        aux2 = sDict[j]
        answer.append(str(aux1.dim)+' '+str(aux1.val)+' '+str(aux2.val))
print answer

['0 1.0 inf', '0 1.0 2.0', '0 1.0 2.0', '0 3.0 4.0', '1 4.0 inf', '1 5.0 6.0']
